7. Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

### 4 Variantes de QDA comparadas 

- **`QDA`**: versión base; calcula e inserta los términos de cada clase con bucles más explícitos.
- **`TensorizedQDA`**: reduce bucles y vectoriza sobre las observaciones.
- **`FasterQDA`**: incrementa la vectorización y el uso de arrays intermedios.
- **`EfficientQDA`**: la más vectorizada; concentra el cálculo en pocas operaciones grandes.

### ¿Qué cambia entre variantes?

Todas implementan la misma regla de decisión QDA; lo que varía es **cómo** se computan, por clase \(k\), los términos: $(x-\mu_k)^{\top}\,\Sigma_k^{-1}\,(x-\mu_k)$ y $\log |(\Sigma_k)|$

haciendo mas precálculo por clase (por ejemplo $\Sigma_k^{-1}$ y $(\log|(\Sigma_k)|)$ y más vectorizaci+on para evitar bucles innecesarios. 

### Qué se observó

- **Exactitud**: prácticamente igual en las cuatro (si aparece alguna diferencia, es de redondeo numérico).
- **Tiempo de cómputo (de más lenta a más rápida):**  
  `QDA` < `TensorizedQDA` < `FasterQDA` < `EfficientQDA`
  
 A medida que se precomputan términos por clase y se reemplazan bucles por operaciones vectorizadas disminuye el trabajo repetido y mejora el tiempo.
- **Memoria**: las variantes más vectorizadas (en especial `EfficientQDA`) pueden usar más RAM por los intermedios; `TensorizedQDA`/`FasterQDA` suelen ofrecer un buen balance tiempo/RAM en tamaños medianos.


### Se condice con lo esperado?
Sí. En QDA el costo típico es:

 **Entrenamiento:** $O(K\,p^3)$ (una inversión/factorización por clase).
 
 **Predicción:** $O(n\,K\,p^2)$. Precomputar lo que no depende de $x$ y 
 
Precomputar lo que **no depende de \(x\)** y **vectorizar** (primero en \(n\), y cuando es posible también en \(K\)) debería acelerar, y eso es lo que se observó.

12. Implementar el modelo TensorizedChol paralelizando sobre clases/observaciones según corresponda. Se recomienda heredarlo de alguna de las implementaciones de QDA_Chol, aunque la elección de cuál de ellas queda a cargo del alumno según lo observado en los benchmarks de puntos anteriores.

In [ ]:
import numpy as np
from scipy.linalg import cholesky, solve_triangular

class TensorizedChol(QDA_Chol3):
    """
    QDA con Cholesky y vectorización (full-batch).
    - Procesa clases × muestras en una sola pasada.
    - Usa solve_triangular para L^{-1}.
    """
    def __init__(self, reg=1e-6, priors=None):
        super().__init__()
        self.reg = float(reg)
        self.user_priors = None if priors is None else np.asarray(priors, float)

    def _fit_params(self, X, y):
        y = np.asarray(y).ravel()
        X = np.asarray(X)
        p, n = X.shape

        # clases y priors
        self.classes_ = np.unique(y)
        K = self.classes_.size
        idx = {c:i for i,c in enumerate(self.classes_)}
        if getattr(self, "log_a_priori", None) is None:
            if self.user_priors is not None:
                pri = self.user_priors / self.user_priors.sum()
            else:
                counts = np.bincount([idx[c] for c in y], minlength=K).astype(float)
                pri = counts / counts.sum()
            self.log_a_priori = np.log(np.clip(pri, 1e-300, None))

        # medias por clase -> (K,p,1)
        means = [X[:, y==c].mean(axis=1, keepdims=True) for c in self.classes_]
        self.tensor_means = np.stack(means, axis=0)

        # Σ_j, Cholesky y L_j^{-1}; log|Σ_j^{-1}| = -2 * sum(log diag(L_j))
        I = np.eye(p)
        l_invs, logdet_inv = [], []
        for c in self.classes_:
            Xc = X[:, y==c]
            cov_j = np.cov(Xc, bias=True) + self.reg * I   # bias=True para alinear con el TP
            L_j  = cholesky(cov_j, lower=True, check_finite=False)
            L_inv_j = solve_triangular(L_j, I, lower=True, check_finite=False)
            l_invs.append(L_inv_j)
            logdet_inv.append(-2.0 * np.log(np.diag(L_j)).sum())
        self.tensor_L_inv = np.stack(l_invs, axis=0)
        self.logdet_inv   = np.asarray(logdet_inv)

    def _scores_block(self, Xb):
        # y = L^{-1}(x - μ) ; quad = ||y||^2
        unbiased = Xb[None,:,:] - self.tensor_means      # (K,p,nb)
        Y = self.tensor_L_inv @ unbiased                 # (K,p,nb)
        quad = np.sum(Y*Y, axis=1)                       # (K,nb)
        return self.log_a_priori[:,None] + 0.5*self.logdet_inv[:,None] - 0.5*quad

    def decision_function(self, X):
        X = np.asarray(X)                   # full-batch siempre
        return self._scores_block(X)        # (K,n)

    def predict(self, X):
        scores = self.decision_function(X)
        return self.classes_[np.argmax(scores, axis=0)]